# 05 - Quantitative Analysis (TA-Lib + PyNance)
Perform Task 2's quantitative deep-dive: load OHLCV data, compute TA-Lib indicators, add PyNance metrics, and visualize the signals.

**References**:
- TA-Lib Function Reference: https://mrjbq7.github.io/ta-lib/func_groups/
- PyNance Technical Indicators: https://github.com/mrtazz/pynance

In [ ]:
# !pip install -r ../requirements.txt
from pathlib import Path
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

import talib
import pynance.tech as ptech

repo_root = Path.cwd().resolve().parent
if repo_root.name == 'notebooks':
    repo_root = repo_root.parent
if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))

from src.data_loader import load_price_csv

In [ ]:
ticker = 'AAPL'  # swap to AMZN/GOOG/NVDA/etc. as needed
price_path = repo_root / 'data' / f'{ticker}.csv'
price = load_price_csv(price_path)
price['Date'] = pd.to_datetime(price['Date'])
price = price.set_index('Date').sort_index()
price.index = price.index.tz_localize(None)

price['simple_return'] = price['Close'].pct_change()
price['log_return'] = np.log(price['Close'] / price['Close'].shift(1))
price['volume_ma_5'] = price['Volume'].rolling(window=5, min_periods=1).mean()
print(f'Loaded {len(price):,} rows from {price_path.name}')

## Data Preview

In [ ]:
display(price.head())

## TA-Lib Indicators

In [ ]:
close = price['Close'].values
price['TA_SMA_20'] = talib.SMA(close, timeperiod=20)
price['TA_SMA_50'] = talib.SMA(close, timeperiod=50)
price['TA_RSI_14'] = talib.RSI(close, timeperiod=14)
macd, macd_signal, macd_hist = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
price['TA_MACD'] = macd
price['TA_MACD_signal'] = macd_signal
price['TA_MACD_hist'] = macd_hist
bb_upper, bb_mid, bb_lower = talib.BBANDS(close, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
price['TA_BB_upper'] = bb_upper
price['TA_BB_mid'] = bb_mid
price['TA_BB_lower'] = bb_lower
display(price[['Close', 'TA_SMA_20', 'TA_SMA_50', 'TA_RSI_14', 'TA_MACD', 'TA_BB_upper']].tail())

## PyNance Metrics

In [ ]:
pn_ret = ptech.ret(price[['Close']], selection='Close')
pn_vol = ptech.volatility(price[['Close']], selection='Close', n_sessions=20)
price = price.join(pn_ret.rename(columns={'Return': 'PN_Return'}))
price = price.join(pn_vol.rename(columns={'Risk': 'PN_Risk_20'}))
price['PN_Return'] = price['PN_Return'].fillna(0)
display(price[['PN_Return', 'PN_Risk_20']].describe())

## Visualize price vs. indicators

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)
axes[0].plot(price.index, price['Close'], label=f'{ticker} Close', color='black', linewidth=1)
axes[0].plot(price.index, price['TA_SMA_20'], label='SMA 20', color='steelblue')
axes[0].plot(price.index, price['TA_SMA_50'], label='SMA 50', color='orange')
axes[0].fill_between(price.index, price['TA_BB_upper'], price['TA_BB_lower'], color='gray', alpha=0.15, label='Bollinger Bands')
axes[0].set_ylabel('Price (USD)')
axes[0].legend(loc='upper left')

axes[1].plot(price.index, price['TA_RSI_14'], color='purple')
axes[1].axhline(70, color='red', linestyle='--', linewidth=0.8)
axes[1].axhline(30, color='green', linestyle='--', linewidth=0.8)
axes[1].set_ylabel('RSI')
axes[1].set_title('RSI (14) with Overbought/Oversold levels')

axes[2].plot(price.index, price['PN_Return'], label='PyNance Return (1 session)', color='teal', alpha=0.7)
axes[2].plot(price.index, price['PN_Risk_20'], label='PyNance Risk (20 sessions)', color='firebrick', alpha=0.7)
axes[2].set_ylabel('Return / Risk')
axes[2].legend(loc='upper left')
axes[2].set_title('PyNance metrics complement TA-Lib outputs')

plt.tight_layout()

_Next: extend this notebook with additional financial metrics (Sharpe/Sortino, ATR, VWAP) and export key plots for the dashboard deliverable._